In [ ]:
# Example submission notebook for the midterm-project in the
# Brown Data Science DATA2040 Deep Learning course. 
# Please visit dsi.brown.edu for information on the Data Sciene Masters Program.
# The web page for this course is data2040.github.io 
# Authors: Kaiwen Yang, Dan Potter

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
import json
import tensorflow as tf
from functools import partial

import tensorflow.keras as keras
import keras
from keras.layers import Dense, Dropout, Input, MaxPooling2D, ZeroPadding2D, Conv2D, Flatten
from keras.models import Sequential, Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.layers import MaxPool2D, AveragePooling2D, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

from zipfile import ZipFile
import time
from datetime import timedelta
from io import BytesIO

# Image manipulation.
import PIL.Image

import pickle
import os

import random


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_labels = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')

In [ ]:
### https://www.kaggle.com/grantwiersum/cassava-disease-tfrecord-training
import glob
csv_df = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')
train_dir = '/kaggle/input/cassava-leaf-disease-classification/train_tfrecords/*'
test_dir = '/kaggle/input/cassava-leaf-disease-classification/test_tfrecords/*'
train_list = glob.glob(train_dir)
test_list = glob.glob(test_dir)
print("train: " + str(len(train_list)) + "\ntest: " + str(len(test_list)))

In [ ]:
labels = train_labels
labels.head()

In [ ]:
labels['label']=labels['label'].astype(str)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./225,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.3,
    horizontal_flip=True,)


test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_dir = '/kaggle/input/cassava-leaf-disease-classification/train_images/'

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe=labels,
      directory=train_dir,
      subset='training',
      x_col="image_id",
      y_col="label",
      shuffle=True,
      target_size=(150,150),
      batch_size=32,
      class_mode='categorical')

valid_generator = train_datagen.flow_from_dataframe(dataframe=labels,
      directory=train_dir,
      subset='validation',
      x_col="image_id",
      y_col="label",
      shuffle=True,
      target_size=(150,150),
      batch_size=32,
      class_mode='categorical')

In [ ]:
# setup tensorboard, directories
!rm -rf ./logs
!mkdir ./logs/
!mkdir ./logs/data2040_midterm_project



log_dir="./logs/data2040_midterm_project/"
def tensorboard_callback(exp_name):
  return tf.keras.callbacks.TensorBoard(log_dir=log_dir + exp_name, profile_batch=0, histogram_freq=1)
# launch tensorboard with specific directory
%load_ext tensorboard
%tensorboard --logdir logs/data2040_midterm_project


In [ ]:
import keras
from keras.layers import Dense, Dropout, Input, MaxPooling2D, ZeroPadding2D, Conv2D, Flatten
from keras.models import Sequential, Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from keras.utils import to_categorical
from tensorflow.keras import regularizers

from tensorflow.keras.layers import MaxPool2D, AveragePooling2D, GlobalAveragePooling2D
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

from zipfile import ZipFile
import time
from datetime import timedelta
from io import BytesIO

# Image manipulation.
import PIL.Image

import pickle
import os

import random

In [ ]:
def make_model_1():
    inputs = Input(shape = (150,150,3))
    model = Conv2D(filters=32, kernel_size=(3, 3),activation='softmax', input_shape=(150,150,3))(inputs)
    model = MaxPool2D(pool_size=(2, 2))(model)
    model = Conv2D(filters=64, kernel_size=(3, 3), activation='softmax')(model)
    model = MaxPool2D(pool_size=(2, 2))(model)
    model = Conv2D(filters=128, kernel_size=(3, 3), activation='softmax')(model)
    model = MaxPool2D(pool_size=(2, 2))(model)
    model = Flatten()(model)
    model = Dense(512, activation = "relu")(model)
    out = Dense(5, activation = 'softmax')(model)
    model = Model(inputs=inputs, outputs=out)
    
    model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(),
              metrics=['acc'])
    return model

In [ ]:
model = make_model_1()
print(model.summary())


In [ ]:
history = model.fit(train_generator,steps_per_epoch=20,
                    validation_data=valid_generator,validation_steps=20,
                    callbacks=[tensorboard_callback('baseline')], epochs=50
)
#EarlyStopping(monitor='val_acc',patience=10)

In [ ]:
# save model
from keras.models import load_model
import os
def save_model(model, name):
  model_name = '{}.h5'.format(name)
  save_dir = os.path.join(os.getcwd(), 'saved_models')
  
  # Save model and weights
  if not os.path.isdir(save_dir):
      os.makedirs(save_dir)
  model_path = os.path.join(save_dir, model_name)
  model.save(model_path)
  print('Saved trained model at %s ' % model_path)

In [ ]:
save_model(model, 'baseline_model')

In [ ]:
# Make sure to save the model you trained to /kaggle/working! 
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import smart_resize
## Load model. Use "load_weights" if you only save your model weights.
model = keras.models.load_model("./saved_models/baseline_model.h5")

preds = []
sample_sub = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv')

for image in sample_sub.image_id:
    img = keras.preprocessing.image.load_img('/kaggle/input/cassava-leaf-disease-classification/test_images/' + image)
    #
    # Preprocess image here (rescale, etc. - you might need to use parameters you determined during training)
    #
    img = img_to_array(img)
    img = smart_resize(img, (150,150))
    img = tf.reshape(img, (-1, 150, 150, 3))
    
    # Now apply your model and save your prediction:
    prediction = model.predict(img)
    preds.append(np.argmax(prediction))
    
    # Blind-Monkey Model
    # This is horrible possible baseline model.  You can improve it by
    # putting all of p's mass on the most commonly occuring class.
    # Question: if you set p to the actual class label distribution, on average,  
    # will you get the same result, a better result or a worse result? 
    # preds.append(np.random.choice(5, p=[.2, .2, .2, .2, .2])) 

my_submission = pd.DataFrame({'image_id': sample_sub.image_id, 'label': preds})
my_submission.to_csv('/kaggle/working/submission.csv', index=False)